# LangGraph and LangSmith - Agentic RAG Powered by LangChain

In the following notebook we'll complete the following tasks:

- 🤝 Breakout Room #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Creating our Tool Belt
  4. Creating Our State
  5. Creating and Compiling A Graph!
  

- 🤝 Breakout Room #2:
  1. Creating an Evaluation Dataset
  2. Adding Evaluators
  3. Evaluating

# 🤝 Breakout Room #1

## LangGraph - Building Cyclic Applications with LangChain

LangGraph is a tool that leverages LangChain Expression Language to build coordinated multi-actor and stateful applications that includes cyclic behaviour.

### Why Cycles?

In essence, we can think of a cycle in our graph as a more robust and customizable loop. It allows us to keep our application agent-forward while still giving the powerful functionality of traditional loops.

Due to the inclusion of cycles over loops, we can also compose rather complex flows through our graph in a much more readable and natural fashion. Effetively allowing us to recreate appliation flowcharts in code in an almost 1-to-1 fashion.

### Why LangGraph?

Beyond the agent-forward approach - we can easily compose and combine traditional "DAG" (directed acyclic graph) chains with powerful cyclic behaviour due to the tight integration with LCEL. This means it's a natural extension to LangChain's core offerings!

## Task 1:  Dependencies

We'll first install all our required libraries.

In [ ]:
!pip install -qU langchain langchain_openai langgraph arxiv duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.1/269.1 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.4/262.4 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.6 MB/s eta 0:0

## Task 2: Environment Variables

We'll want to set both our OpenAI API key and our LangSmith environment variables.

In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key:··········


In [ ]:
from uuid import uuid4

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"AIE1 - LangGraph - {uuid4().hex[0:8]}"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangSmith API Key: ")

LangSmith API Key: ··········


## Task 3: Creating our Tool Belt

As is usually the case, we'll want to equip our agent with a toolbelt to help answer questions and add external knowledge.

There's a tonne of tools in the [LangChain Community Repo](https://github.com/langchain-ai/langchain/tree/master/libs/community/langchain_community/tools) but we'll stick to a couple just so we can observe the cyclic nature of LangGraph in action!

We'll leverage:

- [Duck Duck Go Web Search](https://github.com/langchain-ai/langchain/tree/master/libs/community/langchain_community/tools/ddg_search)
- [Arxiv](https://github.com/langchain-ai/langchain/tree/master/libs/community/langchain_community/tools/arxiv)

####🏗️ Activity #1:

Please add the tools to use into our toolbelt.

> NOTE: Each tool in our toolbelt should be a method.

In [ ]:
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun
from langchain_community.tools.arxiv.tool import ArxivQueryRun

tool_belt = [
    DuckDuckGoSearchRun(),
    ArxivQueryRun(),
]

### Actioning with Tools

Now that we've created our tool belt - we need to create a process that will let us leverage them when we need them.

We'll use the built-in [`ToolExecutor`](https://github.com/langchain-ai/langgraph/blob/main/langgraph/prebuilt/tool_executor.py) to do so.

In [ ]:
from langgraph.prebuilt import ToolExecutor

tool_executor = ToolExecutor(tool_belt)

### Model

Now we can set-up our model! We'll leverage the familiar OpenAI model suite for this example - but it's not *necessary* to use with LangGraph. LangGraph supports all models - though you might not find success with smaller models - as such, they recommend you stick with:

- OpenAI's GPT-3.5 and GPT-4
- Anthropic's Claude
- Google's Gemini

> NOTE: Because we're leveraging the OpenAI function calling API - we'll need to use OpenAI *for this specific example* (or any other service that exposes an OpenAI-style function calling API.

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0)

Now that we have our model set-up, let's "put on the tool belt", which is to say: We'll bind our LangChain formatted tools to the model in an OpenAI function calling format.

In [ ]:
from langchain_core.utils.function_calling import convert_to_openai_function

functions = [convert_to_openai_function(t) for t in tool_belt]
model = model.bind_functions(functions)

#### ❓ Question #1:

How does the model determine which tool to use?

**The tools that the model uses are specified in the tool_belt variable that is passed to the Tool_Executor**

## Putting the State in Stateful

Earlier we used this phrasing:

`coordinated multi-actor and stateful applications`

So what does that "stateful" mean?

To put it simply - we want to have some kind of object which we can pass around our application that holds information about what the current situation (state) is. Since our system will be constructed of many parts moving in a coordinated fashion - we want to be able to ensure we have some commonly understood idea of that state.

LangGraph leverages a `StatefulGraph` which uses an `AgentState` object to pass information between the various nodes of the graph.

There are more options than what we'll see below - but this `AgentState` object is one that is stored in a `TypedDict` with the key `messages` and the value is a `Sequence` of `BaseMessages` that will be appended to whenever the state changes.

Let's think about a simple example to help understand exactly what this means (we'll simplify a great deal to try and clearly communicate what state is doing):

1. We initialize our state object:
  - `{"messages" : []}`
2. Our user submits a query to our application.
  - New State: `HumanMessage(#1)`
  - `{"messages" : [HumanMessage(#1)}`
3. We pass our state object to an Agent node which is able to read the current state. It will use the last `HumanMessage` as input. It gets some kind of output which it will add to the state.
  - New State: `AgentMessage(#1, additional_kwargs {"function_call" : "WebSearchTool"})`
  - `{"messages" : [HumanMessage(#1), AgentMessage(#1, ...)]}`
4. We pass our state object to a "conditional node" (more on this later) which reads the last state to determine if we need to use a tool - which it can determine properly because of our provided object!

In [ ]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
  messages: Annotated[Sequence[BaseMessage], operator.add]

## It's Graphing Time!

Now that we have state, and we have tools, and we have an LLM - we can finally start making our graph!

Let's take a second to refresh ourselves about what a graph is in this context.

Graphs, also called networks in some circles, are a collection of connected objects.

The objects in question are typically called nodes, or vertices, and the connections are called edges.

Let's look at a simple graph.

![image](https://i.imgur.com/2NFLnIc.png)

Here, we're using the coloured circles to represent the nodes and the yellow lines to represent the edges. In this case, we're looking at a fully connected graph - where each node is connected by an edge to each other node.

If we were to think about nodes in the context of LangGraph - we would think of a function, or an LCEL runnable.

If we were to think about edges in the context of LangGraph - we might think of them as "paths to take" or "where to pass our state object next".

Let's create some nodes and expand on our diagram.

> NOTE: Due to the tight integration with LCEL - we can comfortably create our nodes in an async fashion!

In [ ]:
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage

def call_model(state):
  messages = state["messages"]
  response = model.invoke(messages)
  return {"messages" : [response]}

def call_tool(state):
  last_message = state["messages"][-1]

  action = ToolInvocation(
      tool=last_message.additional_kwargs["function_call"]["name"],
      tool_input=json.loads(
          last_message.additional_kwargs["function_call"]["arguments"]
      )
  )

  response = tool_executor.invoke(action)

  function_message = FunctionMessage(content=str(response), name=action.tool)

  return {"messages" : [function_message]}

Now we have two total nodes. We have:

- `call_model` is a node that will...well...call the model
- `call_tool` is a node which will call a tool

Let's start adding nodes! We'll update our diagram along the way to keep track of what this looks like!


In [ ]:
from langgraph.graph import StateGraph, END

workflow = StateGraph(AgentState)

workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)

Let's look at what we have so far:

![image](https://i.imgur.com/md7inqG.png)

Next, we'll add our entrypoint. All our entrypoint does is indicate which node is called first.

In [ ]:
workflow.set_entry_point("agent")

![image](https://i.imgur.com/wNixpJe.png)

Now we want to build a "conditional edge" which will use the output state of a node to determine which path to follow.

We can help conceptualize this by thinking of our conditional edge as a conditional in a flowchart!

Notice how our function simply checks if there is a "function_call" kwarg present.

Then we create an edge where the origin node is our agent node and our destination node is *either* the action node or the END (finish the graph).

It's important to highlight that the dictionary passed in as the third parameter (the mapping) should be created with the possible outputs of our conditional function in mind. In this case `should_continue` outputs either `"end"` or `"continue"` which are subsequently mapped to the action node or the END node.

In [ ]:
def should_continue(state):
  last_message = state["messages"][-1]

  if "function_call" not in last_message.additional_kwargs:
    return "end"

  return "continue"

workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue" : "action",
        "end" : END
    }
)

Let's visualize what this looks like.

![image](https://i.imgur.com/8ZNwKI5.png)

Finally, we can add our last edge which will connect our action node to our agent node. This is because we *always* want our action node (which is used to call our tools) to return its output to our agent!

In [ ]:
workflow.add_edge("action", "agent")

Let's look at the final visualization.

![image](https://i.imgur.com/NWO7usO.png)

All that's left to do now is to compile our workflow - and we're off!

In [ ]:
app = workflow.compile()

#### ❓ Question #2:

Is there any specific limit to how many times we can cycle?

**No I don´t think there is a limit, at least a limit set in the code**

If not, how could we impose a limit to the number of cycles?

**You could set a limit by controlling the number of cycles in the code. Given that the graph is directed it could be set on any of the three nodes. Probably the conditional node would be the natural one to use**

## Using Our Graph

Now that we've created and compiled our graph - we can call it *just as we'd call any other* `Runnable`!

Let's try out a few examples to see how it fairs:

In [ ]:
from langchain_core.messages import HumanMessage

inputs = {"messages" : [HumanMessage(content="What is RAG in the context of Large Language Models? When did it break onto the scene?")]}

app.invoke(inputs)

{'messages': [HumanMessage(content='What is RAG in the context of Large Language Models? When did it break onto the scene?'),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"RAG in the context of Large Language Models"}', 'name': 'duckduckgo_search'}}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 171, 'total_tokens': 196}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_4f0b692a78', 'finish_reason': 'function_call', 'logprobs': None}),
  FunctionMessage(content='The beauty of RAG lies in its ability to enable a language model to draw upon and leverage your own data to generate responses. While base models are traditionally trained on specific, point-in-time data, ensuring their effectiveness in performing tasks and adapting to the desired domain, they can struggle when faced with newer or current data. RAG stands for R etrieval- A ugmented G eneration. RAG enables large language models (LLM) to access and uti

Let's look at what happened:

1. Our state object was populated with our request
2. The state object was passed into our entry point (agent node) and the agent node added an `AIMessage` to the state object and passed it along the conditional edge
3. The conditional edge received the state object, found the "function_call" `additional_kwarg`, and sent the state object to the action node
4. The action node added the response from the OpenAI function calling endpoint to the state object and passed it along the edge to the agent node
5. The agent node added a response to the state object and passed it along the conditional edge
6. The conditional edge received the state object, could not find the "function_call" `additional_kwarg` and passed the state object to END where we see it output in the cell above!

Now let's look at an example that shows a multiple tool usage - all with the same flow!

In [ ]:
inputs = {"messages" : [HumanMessage(content="What is QLoRA in Machine Learning? Are their any papers that could help me understand? Once you have that information, can you look up the bio of the first author on the QLoRA paper?")]}

app.invoke(inputs)

{'messages': [HumanMessage(content='What is QLoRA in Machine Learning? Are their any papers that could help me understand? Once you have that information, can you look up the bio of the first author on the QLoRA paper?'),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"QLoRA in Machine Learning"}', 'name': 'arxiv'}}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 193, 'total_tokens': 212}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'function_call', 'logprobs': None}),
  FunctionMessage(content="Published: 2023-05-23\nTitle: QLoRA: Efficient Finetuning of Quantized LLMs\nAuthors: Tim Dettmers, Artidoro Pagnoni, Ari Holtzman, Luke Zettlemoyer\nSummary: We present QLoRA, an efficient finetuning approach that reduces memory usage\nenough to finetune a 65B parameter model on a single 48GB GPU while preserving\nfull 16-bit finetuning task performance. QLoRA backpropagates gradients 

####🏗️ Activity #2:

Please write out the steps the agent took to arrive at the correct answer.

** 1. The state object is populated with the user's request about QLoRA in Machine Learning, asking for relevant papers and then the bio of the first author of the QLoRA paper.**

**2. The state object is passed into the entry point (agent node), which identifies the need to retrieve information about QLoRA from a scholarly database. The agent node adds an AIMessage to the state object with a function_call to search ArXiv for "QLoRA in Machine Learning" and passes it along the conditional edge.**

**3. The conditional edge receives the state object, finds the "function_call" additional_kwarg, and directs the state object to the action node tasked with executing the ArXiv search.**

**4. The action node conducts the ArXiv search, adds the response detailing QLoRA-related papers to the state object, and passes it along the edge back to the agent node.**

**5.Having retrieved the information on QLoRA papers, the agent node now addresses the second part of the request: it adds another AIMessage to the state object with a function_call for a "duckduckgo_search" to find the bio of Tim Dettmers, the first author mentioned in the QLoRA papers, and passes it along the conditional edge.**

**6. The conditional edge again finds a "function_call" additional_kwarg and sends the state object to the corresponding action node for execution.**

**7. The action node performs the DuckDuckGo search for Tim Dettmers's bio, adds the search results to the state object, and returns it along the edge to the agent node.**

**8. The agent node compiles the gathered information into a response, summarizing QLoRA's significance in Machine Learning, the key details of the relevant paper, and Tim Dettmers's bio. This compiled response is added to the state object.**

**9. The conditional edge receives the state object for the last time, does not find any "function_call" additional_kwarg because the request has been fully addressed, and passes the state object to END, resulting in the output presented above.**

### Pre-processing for LangSmith

To do a little bit more preprocessing, let's wrap our LangGraph agent in a simple chain.

In [ ]:
def convert_inputs(input_object):
  return {"messages" : [HumanMessage(content=input_object["question"])]}

def parse_output(input_state):
  return input_state["messages"][-1].content

agent_chain = convert_inputs | app | parse_output

In [ ]:
agent_chain.invoke({"question" : "What is RAG?"})

"Retrieval-augmented generation (RAG) is an AI framework for improving the quality of language model-generated responses by grounding the model on external sources of knowledge to supplement the model's internal representation of information. RAG enhances the accuracy and reliability of generative AI models by fetching facts from external sources. It combines the strengths of retrieval-based and generative models to revolutionize AI systems."

# 🤝 Breakout Room #2

## Task 1: Creating An Evaluation Dataset

Just as we saw last week, we'll want to create a dataset to test our Agent's ability to answer questions.

In order to do this - we'll want to provide some questions and some answers. Let's look at how we can create such a dataset below.

```python
questions = [
    "What optimizer is used in QLoRA?",
    "What data type was created in the QLoRA paper?",
    "What is a Retrieval Augmented Generation system?",
    "Who authored the QLoRA paper?",
    "What is the most popular deep learning framework?",
    "What significant improvements does the LoRA system make?"
]

answers = [
    {"must_mention" : ["paged", "optimizer"]},
    {"must_mention" : ["NF4", "NormalFloat"]},
    {"must_mention" : ["ground", "context"]},
    {"must_mention" : ["Tim", "Dettmers"]},
    {"must_mention" : ["PyTorch", "TensorFlow"]},
    {"must_mention" : ["reduce", "parameters"]},
]
```

####🏗️ Activity #3:

Please create a dataset in the above format with at least 5 questions.

In [ ]:
questions = [
    "What does QLoRA stand for in the context of machine learning?",
    "What innovations does QLoRA introduce to save memory without sacrificing performance?",
    "How does the Paged Optimizers feature work in QLoRA?",
    "What is the role of Double Quantization in QLoRA?",
    "Describe the experimental setup used to compare QLoRA with 16-bit adapter-finetuning and full-finetuning."
]

answers = [
    {"must_mention" : ["Quantized Low-Rank Adapters"]},
    {"must_mention" : ["4-bit NormalFloat (NF4)", "Double Quantization", "Paged Optimizers"]},
    {"must_mention" : ["NVIDIA unified memory", "automatic page-to-page transfers", "GPU occasionally runs out-of-memory"]},
    {"must_mention" : ["quantizing the quantization constants", "additional memory savings"]},
    {"must_mention" : ["three architectures", "GLUE with RoBERTa-large", "Super-NaturalInstructions with T5", "5-shot MMLU"]}
]


Now we can add our dataset to our LangSmith project using the following code which we saw last Thursday!

In [ ]:
from langsmith import Client

client = Client()
dataset_name = f"Retrieval Augmented Generation - Evaluation Dataset - {uuid4().hex[0:8]}"

dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Questions about the QLoRA Paper to Evaluate RAG over the same paper."
)

client.create_examples(
    inputs=[{"question" : q} for q in questions],
    outputs=answers,
    dataset_id=dataset.id,
)

#### ❓ Question #3:

How are the correct answers associated with the questions?

> NOTE: Feel free to indicate if this is problematic or not

**The correct answers are associated with the questions based on specific details or topics covered in the document about QLoRA and its functionalities. Here’s a brief explanation of how each answer is related to its corresponding question, and whether this association might be problematic:**

**1. What does QLoRA stand for in the context of machine learning?**

**The answer "Quantized Low-Rank Adapters" directly defines the acronym QLoRA. This association is straightforward and not problematic as it provides a clear definition.**

**2. What innovations does QLoRA introduce to save memory without sacrificing performance?**

**The answer mentions "4-bit NormalFloat (NF4), Double Quantization, Paged Optimizers," which are specific innovations introduced by QLoRA to reduce memory usage while maintaining performance. This association is precise and informative, closely tying the innovations to the question's focus.**

**3. How does the Paged Optimizers feature work in QLoRA?**

**The answer details the use of "NVIDIA unified memory, automatic page-to-page transfers, GPU occasionally runs out-of-memory," explaining how Paged Optimizers manage memory resources. This association is appropriate, as it explains a technical feature directly related to the question, though it may require some technical background to fully understand.**

**4. What is the role of Double Quantization in QLoRA?**

**The answer highlights "quantizing the quantization constants, additional memory savings," which directly addresses the role of Double Quantization in optimizing memory usage. This is a clear and specific association, highlighting a key component of QLoRA's memory efficiency strategy.**

**5. Describe the experimental setup used to compare QLoRA with 16-bit adapter-finetuning and full-finetuning.**

**The answer includes "three architectures, GLUE with RoBERTa-large, Super-NaturalInstructions with T5, 5-shot MMLU," providing an overview of the experimental setup used in comparisons. This association is a bit broad, covering several aspects of the setup, but it directly answers the question by outlining the framework for comparison.**

**Overall, the associations between questions and answers are well-defined and directly relevant to the questions posed. While some answers require a technical understanding of the topics discussed, this alignment is not inherently problematic. It ensures that the dataset can effectively test an agent's ability to comprehend and recall specific details from the document regarding QLoRA.**

## Task 2: Adding Evaluators

Now we can add a custom evaluator to see if our responses contain the expected information.

We'll be using a fairly naive exact-match process to determine if our response contains specific strings.

In [ ]:
from langsmith.evaluation import EvaluationResult, run_evaluator

@run_evaluator
def must_mention(run, example) -> EvaluationResult:
    prediction = run.outputs.get("output") or ""
    required = example.outputs.get("must_mention") or []
    score = all(phrase in prediction for phrase in required)
    return EvaluationResult(key="must_mention", score=score)

#### ❓ Question #4:

What are some ways you could improve this metric as-is?

> NOTE: Alternatively you can suggest where gaps exist in this method.

**This metric looks for a literal match between the "must mention" key and the result, which I find too narrow. A better metric could be a similarity measure between the key and the response**

Now that we have created our custom evaluator - let's initialize our `RunEvalConfig` with it, and a few others:

- `"criteria"` includes the default criteria which, in this case, means "helpfulness"
- `"cot_qa"` includes a criteria that bases whether or not the answer is correct by utilizing a Chain of Thought prompt and the provided context to determine if the response is correct or not.

In [ ]:
from langchain.smith import RunEvalConfig, run_on_dataset

eval_config = RunEvalConfig(
    custom_evaluators=[must_mention],
    evaluators=[
        "criteria",
        "cot_qa",
    ],
)

Task 3: Evaluating

All that is left to do is evaluate our agent's response!

In [ ]:
client.run_on_dataset(
    dataset_name=dataset_name,
    llm_or_chain_factory=agent_chain,
    evaluation=eval_config,
    verbose=True,
    project_name=f"RAG Pipeline - Evaluation - {uuid4().hex[0:8]}",
    project_metadata={"version": "1.0.0"},
)

View the evaluation results for project 'RAG Pipeline - Evaluation - 31420db9' at:
https://smith.langchain.com/o/ec9be7fd-760d-5000-adc7-0519d0832356/datasets/d49d7f0e-7dae-4189-a963-2b1ceea58cee/compare?selectedSessions=279f23cd-5927-40db-b4fd-f49b792f02d6

View all tests for Dataset Retrieval Augmented Generation - Evaluation Dataset - 9a5af534 at:
https://smith.langchain.com/o/ec9be7fd-760d-5000-adc7-0519d0832356/datasets/d49d7f0e-7dae-4189-a963-2b1ceea58cee
[------------------------------------------------->] 5/5
 Experiment Results:
        feedback.helpfulness  feedback.COT Contextual Accuracy feedback.must_mention error  execution_time                                run_id
count                   5.00                              5.00                     5     0            5.00                                     5
unique                   NaN                               NaN                     2     0             NaN                                     5
top                  

{'project_name': 'RAG Pipeline - Evaluation - 31420db9',
 'results': {'7621d9bf-a840-4aa1-a941-b28f2310dfea': {'input': {'question': 'What does QLoRA stand for in the context of machine learning?'},
   'feedback': [EvaluationResult(key='helpfulness', score=1, value='Y', comment="The criterion for this task is the helpfulness of the submission. \n\nThe submission provides a clear and concise definition of QLoRA in the context of machine learning, which is the question asked in the input. It not only provides the full form of the acronym but also gives a brief explanation of what it does. This information is helpful for someone who wants to understand what QLoRA is. \n\nThe submission is insightful as it provides additional information about how QLoRA works, stating that it combines different techniques to optimize a machine's limited memory without compromising model performance. This gives the reader a deeper understanding of the concept.\n\nThe submission is appropriate as it directly